<a href="https://colab.research.google.com/github/licTomasPerez/Spin-Chains-/blob/main/expect_sxsx_sysy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
## las matrices sí están bien cargadas

N = 5
id2 = qeye(2)
sx = sigmax()
sy = sigmay()
sz = sigmaz()

sx_list = []
sy_list = []
sz_list = []

for n in range(N):
    operator_list = []
    for m in range(N):
        operator_list.append(id2)
    operator_list[n] = sx
    sx_list.append(tensor(operator_list))
        
    operator_list[n] = sy
    sy_list.append(tensor(operator_list))
        
    operator_list[n] = sz
    sz_list.append(tensor(operator_list))

sxsx_list = [sx_list[n]*sx_list[m] for n in range(N) for m in range(N)]
sysy_list = [sy_list[n]*sy_list[m] for n in range(N) for m in range(N)]

print(sysy_list)    

[Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
Qobj data =
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]], Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
Qobj data =
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
Qobj data =
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
Qobj data =
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ...

In [3]:
!pip install matplotlib
!pip install qutip

     |████████████████████████████████| 14.7 MB 4.9 MB/s 


In [4]:
!pip install numpy

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from qutip import *

In [6]:
### XYZ Heisenberg model

solver = "mc"   
N = 5            # número de spines

h  = 1.0 * 2 * np.pi * np.ones(N) 
Jz = 0.1 * 2 * np.pi * np.ones(N)
Jx = 0.15 * 2 * np.pi * np.ones(N)
Jy = 0.25 * 2 * np.pi * np.ones(N)

In [ ]:
def hamiltonian(N, Jx, Jy, Jz, h, psi0, tlist, gamma, solver):
    
    id2 = qeye(2)
    sx = sigmax()
    sy = sigmay()
    sz = sigmaz()

    sx_list = []
    sy_list = []
    sz_list = []

    for n in range(N):
        operator_list = []
        for m in range(N):
            operator_list.append(id2)
        operator_list[n] = sx
        sx_list.append(tensor(operator_list))
        
        operator_list[n] = sy
        sy_list.append(tensor(operator_list))
        
        operator_list[n] = sz
        sz_list.append(tensor(operator_list))

    sxsx_list = [sx_list[n]*sx_list[m] for n in range(N) for m in range(N)]
    sysy_list = [sy_list[n]*sy_list[m] for n in range(N) for m in range(N)]
    
    # no es terriblemente eficiente pero funca al menos
    #for n in range(N):
    #  for m in range(N):
    #    sxsx_list.append(sx_list[n]*sx_list[m])
    #    sysy_list.append(sy_list[n]*sy_list[m])
  
    # Constructing the Hamiltonian
    
    H = 0
    
    for n in range(N):
        H += -0.5*h[n]*sz_list[n]
        
    for n in range(N-1):
        H += -0.5 * Jx[n] * sx_list[n] * sx_list[n+1]
        H += -0.5 * Jy[n] * sy_list[n] * sy_list[n+1]
        H += -0.5 * Jz[n] * sz_list[n] * sz_list[n+1]
    
    # collapse operators
    c_op_list = []

    # spin dephasing
    for n in range(N):
        if gamma[n] > 0.0:
            c_op_list.append(np.sqrt(gamma[n]) * sz_list[n])
    
    if solver == "me":
        result_spin_ops = mesolve(H, psi0, tlist, c_op_list, sxsx_list)
    elif solver == "mc":
        ntraj = 200 
        result_spin_ops = mcsolve(H, psi0, tlist, c_op_list, sxsx_list, ntraj)

    return result_spin_ops


In [ ]:
# dephasing rate
gamma = 5 * 10**-10* np.ones(N)

# intial state, first spin in state |1>, the rest in state |0>
psi_list = []
for n in range(2):
    psi_list.append(basis(2,0))
psi_list.append(basis(2,1))
for n in range(2):
    psi_list.append(basis(2,0))
psi0 = tensor(psi_list)

tlist = np.linspace(0, 10, 100)

spin_ops_expt = hamiltonian(N, Jx, Jy, Jz, h, psi0, tlist, gamma, solver)


10.0%. Run time:   0.72s. Est. time left: 00:00:00:06
20.0%. Run time:   1.35s. Est. time left: 00:00:00:05
30.0%. Run time:   1.98s. Est. time left: 00:00:00:04
40.0%. Run time:   2.60s. Est. time left: 00:00:00:03
50.0%. Run time:   3.39s. Est. time left: 00:00:00:03
60.0%. Run time:   4.02s. Est. time left: 00:00:00:02
70.0%. Run time:   4.68s. Est. time left: 00:00:00:02
80.0%. Run time:   5.30s. Est. time left: 00:00:00:01
90.0%. Run time:   5.91s. Est. time left: 00:00:00:00
100.0%. Run time:   6.50s. Est. time left: 00:00:00:00
Total run time:   6.53s


In [ ]:
spin_ops_expt

Result object with mcsolve data.
--------------------------------
expect = True
num_expect = 25, num_collapse = 5, ntraj = 200

In [ ]:
def hamiltonian(N, Jx, Jy, Jz, h, psi0, tlist, gamma, solver):
    
    id2 = qeye(2)
    sx = sigmax()
    sy = sigmay()
    sz = sigmaz()

    sx_list = []
    sy_list = []
    sz_list = []

    for n in range(N):
        operator_list = []
        for m in range(N):
            operator_list.append(id2)
        operator_list[n] = sx
        sx_list.append(tensor(operator_list))
        
        operator_list[n] = sy
        sy_list.append(tensor(operator_list))
        
        operator_list[n] = sz
        sz_list.append(tensor(operator_list))

    sxsx_list = []
    sysy_list = []
    
    # no es terriblemente eficiente pero funca al menos
    for n in range(N):
      for m in range(N):
        sxsx_list.append(sx_list[n]*sx_list[n])
        sysy_list.append(sy_list[n]*sy_list[n])
    return sxsx_list, sysy_list

In [ ]:
# dephasing rate
gamma = 5 * 10**-10* np.ones(N)

# intial state, first spin in state |1>, the rest in state |0>
psi_list = []
for n in range(2):
    psi_list.append(basis(2,0))
psi_list.append(basis(2,1))
for n in range(2):
    psi_list.append(basis(2,0))
psi0 = tensor(psi_list)

tlist = np.linspace(0, 10, 100)

sXsX_list1, sYsY_list2 = hamiltonian(N, Jx, Jy, Jz, h, psi0, tlist, gamma, solver)

In [ ]:
sXsX_list1

[Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]],
 Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]],
 Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]],
 Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0

In [ ]:
sYsY_list2

[Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]],
 Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]],
 Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 0. 0. 1.]],
 Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
 Qobj data =
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0